<a href="https://colab.research.google.com/github/alessiomongoli/Sentiment_Lexicon/blob/main/Utils/DataframeCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def creation_dataframe(vocabulary,coefficents, frequencies):
  word=[]
  valore=[]
  pol=[]
  for key,value in vocabulary.items():
    word.append(key)
    valore.append(frequencies[value])
    pol.append(coefficents[value])
  data=pd.DataFrame(columns=['Token','Frequence','Polarity'])
  data['Token']=word
  data['Frequence']=valore
  data['Polarity']=pol
  return data

In [ ]:
def create_dataframe_price_equal_size(path_category, path_metadata):
  
  df_category = getDF(path_category)
  df_category = df_category.drop(columns=['reviewerID', 'reviewerName', 'helpful', 'summary', 'unixReviewTime', 'reviewTime'])
  df_category = df_category[df_category['overall'] != 3.0]
  df_category = df_category.rename(columns = {'overall':'label'})
  df_category=df_category.dropna().reset_index(drop=True)
  
  df_category = df_category.set_index('asin')
  df_metadata = getDF(path_metadata)
  df_metadata = df_metadata.drop(columns=[ 'title', 'imUrl', 'salesRank', 'categories', 'related','brand', 'description'])
  df_metadata = df_metadata.set_index('asin')
  df_new = df_category.join(df_metadata)
  df_new=df_new.dropna().reset_index(drop=True)
  df_new['price'] = df_new['price'].round(decimals = 0)
  df_new = df_new.astype({'price':'int'})
  df_new = df_new[df_new['price']>0].reset_index(drop=True)
  
  df_new = df_new.sort_values(by = 'price').reset_index(drop=True)
  df_low_price, df_medium_price, df_high_price = np.array_split(df_new, 3)

  df_medium_price=df_medium_price.reset_index(drop=True)
  df_high_price=df_high_price.reset_index(drop=True)
                                     
  return df_low_price, df_medium_price, df_high_price

In [ ]:
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
def compute_statistics_dataframe (df):
  min_price = df['price'].min()
  max_price = df['price'].max()
  mean_price =df['price'].mean()
  number_of_reviews = len(df)
  c = df['reviewText'].apply(lambda x: len(x.split()))
  mean_lenght_review = c.mean()

  return min_price, max_price, mean_price, number_of_reviews, mean_lenght_review